## Homework 02: Unsupervised embedding-based MT
*Note: this homework is based on open materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [3]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [8]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.bin.gz

--2021-04-18 23:17:13--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4504123670 (4.2G) [application/octet-stream]
Saving to: ‘cc.uk.300.bin.gz’

cc.uk.300.bin.gz    100%[===================>]   4.19G  22.6MB/s    in 3m 11s  

2021-04-18 23:20:24 (22.5 MB/s) - ‘cc.uk.300.bin.gz’ saved [4504123670/4504123670]



In [9]:
!gzip -d cc.uk.300.bin.gz

gzip: cc.uk.300.bin already exists; do you wish to overwrite (y or n)? y
y


In [10]:
uk_emb = gensim.models.FastText.load_fasttext_format("cc.uk.300.bin")
uk_emb = uk_emb.wv

In [11]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz

--2021-04-18 23:23:58--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4496459151 (4.2G) [application/octet-stream]
Saving to: ‘cc.ru.300.bin.gz’

cc.ru.300.bin.gz    100%[===================>]   4.19G  23.7MB/s    in 3m 1s   

2021-04-18 23:26:59 (23.7 MB/s) - ‘cc.ru.300.bin.gz’ saved [4496459151/4496459151]



In [12]:
!gzip -d cc.ru.300.bin.gz

gzip: cc.ru.300.bin already exists; do you wish to overwrite (y or n)? y


In [13]:
ru_emb = gensim.models.FastText.load_fasttext_format("cc.ru.300.bin")
ru_emb = ru_emb.wv

In [16]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.8918135166168213),
 ('июнь', 0.8817341327667236),
 ('март', 0.8627270460128784),
 ('ноябрь', 0.850366473197937),
 ('октябрь', 0.8314876556396484),
 ('декабрь', 0.8227221965789795),
 ('январь', 0.8169068098068237),
 ('апрель', 0.7869453430175781),
 ('сентябрь', 0.7696788311004639)]

In [17]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 1.0),
 ('серпеня', 0.882050633430481),
 ('серпені', 0.8710206151008606),
 ('серпен', 0.8673141002655029),
 ('серпень-жовтень', 0.852085530757904),
 ('серпнясерпень', 0.8448044657707214),
 ('лютий-серпень', 0.8408336639404297),
 ('серпентарій', 0.8381907939910889),
 ('серпень.Температури', 0.8358946442604065),
 ('травень-серпень', 0.8319928050041199)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [18]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [19]:
!wget -O ukr_rus.train.txt http://tiny.cc/jfgecz

--2021-04-18 23:35:10--  http://tiny.cc/jfgecz
Resolving tiny.cc (tiny.cc)... 157.245.113.153
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/jfgecz [following]
--2021-04-18 23:35:10--  https://tiny.cc/jfgecz
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt [following]
--2021-04-18 23:35:10--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59351 (58K) [text/plain]
Sa

In [20]:
!wget -O ukr_rus.test.txt http://tiny.cc/6zoeez

--2021-04-18 23:35:11--  http://tiny.cc/6zoeez
Resolving tiny.cc (tiny.cc)... 157.245.113.153
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/6zoeez [following]
--2021-04-18 23:35:11--  https://tiny.cc/6zoeez
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt [following]
--2021-04-18 23:35:11--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12188 (12K) [text/plain]
Savi

In [21]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [22]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [23]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE
mapping = LinearRegression(fit_intercept=True).fit(X_train, Y_train)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [24]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august, topn=10)

[('март', 0.5900105237960815),
 ('ноябрь', 0.5561882257461548),
 ('январь', 0.5516124963760376),
 ('февраль', 0.5507173538208008),
 ('апрель', 0.507216215133667),
 ('июнь', 0.5049359798431396),
 ('август', 0.5006322264671326),
 ('декабрь', 0.49596941471099854),
 ('зима', 0.48768675327301025),
 ('солнцестояние', 0.48156559467315674)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [25]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...] 
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    k = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
        word, value = zip(*ru_emb.most_similar(mapped_vectors[k].reshape(1, -1), topn = topn))
        num_matches += int(ru in word)
        k = k + 1
    precision_val = round(num_matches / len(pairs), 2) 
    return precision_val

In [26]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [27]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [28]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [29]:
print(precision_top1, precision_top5)

0.47 0.67


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [30]:
import numpy as np

In [31]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOUR CODE HERE
    a = X_train.T @ Y_train
    u, s, vh = np.linalg.svd(a)
    # compute orthogonal embedding space mapping
    mapping = u @ vh

    return mapping

In [32]:
W = learn_transform(X_train, Y_train)

In [223]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('март', 0.4410023093223572),
 ('февраль', 0.430755078792572),
 ('ноябрь', 0.4286820888519287),
 ('декабрь', 0.407553493976593),
 ('январь', 0.4010366201400757),
 ('июль', 0.3999900817871094),
 ('октябрь', 0.3791142702102661),
 ('июнь', 0.37881380319595337),
 ('август', 0.3679208755493164),
 ('апрель', 0.35369962453842163)]

In [34]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.52
0.72


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [35]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz

--2021-04-18 23:43:25--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819128 (1.7M) [application/gzip]
Saving to: ‘uk.txt.gz’

uk.txt.gz           100%[===================>]   1.73M  2.33MB/s    in 0.7s    

2021-04-18 23:43:27 (2.33 MB/s) - ‘uk.txt.gz’ saved [1819128/1819128]



In [36]:
!gzip -d ./uk.txt.gz

gzip: ./uk.txt already exists; do you wish to overwrite (y or n)? y


In [215]:
with open('./uk.txt', 'r') as f:
    uk_corpus = f.readlines()

In [216]:
# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]

In [217]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
word_tokenize(sentence_data)
uk_corpus = [word_tokenize(token) for token in uk_corpus]
k = 0
for word in uk_corpus:
  uk_corpus[k] = " ".join(uk_corpus[k])
  k +=1

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [155]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    translated = str(sentence)
    sentence = sentence.split(' ')
    # YOUR CODE GOES HERE
    for word in sentence:
        # find ukrainian embedding for each word in sentence
        ebmdn = uk_emb[word]
        # transform ukrainian embedding vector 
        # find nearest russian word and replace
        russian, _ = zip(*ru_emb.most_similar([np.matmul(ebmdn, W)], topn=1))
        translated = translated.replace(word, russian[0])
    translated = translated.split(' ')
    return " ".join(translated)

In [157]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышь"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [45]:
UNK = 'Missed'

In [219]:
for sent in uk_corpus[::10]:
      print(sent, ' ', translate(sent))

Я вже закінчу коледж , коли ви вернетеся з Америки .   мной уже прилечу колледж , когда мы житьИ со Америку .
Місто бомбардували ворожі літаки .   город бомбардировали вражеские самолёты .
Можливо , я антисоціальний , але це не означає , що я не спілкуюся з людьми .   Врмнойд , мной дерсокий , но это не осоначает , что мной не спілкуюсмной со людьми .
Цього ранку випала роса .   Тогда утро выпала роса .
Біда не приходить одна .   Беда не приходит одна .
Подивися на той дим .   Разуй по тот дым .
Я замовив два гамбургера .   мной заказал два бургера .
Я не хотів нікого образити .   мной не хотел НИКОГО оскорбить .
Гора вкрита снігом .   гора покрыта снегом .
На фотографії в дівчини корона не з золота , а з квітів .   по снимки во дівочини корона не со соолота , а со квоітіво .
У мене є мрія .   Во меня является мечта .
Я приїхав у Японію з Китаю .   мной приехал во мнойпонію со китая. .
На півночі знаходиться Шотландія ; на півдні — Англія ; на заході — Уельс ; і ще далі на заході — Пів